In [ ]:
# This notebook should be run on Google Colab

## Mount GDrive for Collab

In [ ]:
print('Mounting Gdrive')
from google.colab import drive
drive.mount('/content/drive')

import sys
import os
sys.path.append('/content/drive/MyDrive/')

## Lib

In [28]:
import os
import tensorflow as tf
from tensorflow.keras.models import load_model


from mlflow import MlflowClient
import mlflow.tensorflow

## Vars

In [29]:
model_path = "../../storage/models/generated_model_efficientnetv2_21k_finetuned_1k_v1.keras"
epochs = 5
learning_rate = 1e-4

In [30]:
train_dataset_path = "../../storage/datas/tf_datasets/train_dataset"
test_dataset_path = "../../storage/datas/tf_datasets/test_dataset"
val_dataset_path = "../../storage/datas/tf_datasets/val_dataset"
tracking_uri = "../../storage/mlruns"
model_checkpoint_url = "../../storage/model_checkpoint"

## Load model & Datasets

### Model

In [31]:
# Load Model
model = tf.keras.models.load_model(model_path)
#model = keras.saving.load_model(model_path, custom_objects=None, compile=False, safe_mode=True)

# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=3)])

ValueError: Only instances of `keras.Layer` can be added to a Sequential model. Received: <tensorflow_hub.keras_layer.KerasLayer object at 0x7fc3668afc40> (of type <class 'tensorflow_hub.keras_layer.KerasLayer'>)

### Datasets

In [ ]:
# Load datasets
train_datas = tf.data.experimental.load(train_dataset_path)
val_datas = tf.data.experimental.load(val_dataset_path)

## Fit

In [ ]:
# Callbacks
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    os.path.join(model_checkpoint_url, "model_{epochs:02d}_{val_loss:.2f}.weights.h5"),
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    mode='auto',
    save_freq='epoch',
    initial_value_threshold=None)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=3,
    verbose=1,
    mode='auto',
    baseline=None,
    restore_best_weights=True,
    start_from_epoch=5)

readuce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=5,
    verbose=1,
    mode='auto',
    min_delta=0.0001,
    cooldown=0)

### Mlflow Client


In [ ]:
# Start mlflow client
mlflow.set_tracking_uri(tracking_uri)
client = MlflowClient()
mlflow.tensorflow.autolog()

In [ ]:
history = model.fit(
    train_datas,
    validation_data = val_datas,
    epochs = args.epochs,
    initial_epoch = 0,
    callbacks=[model_checkpoint, early_stopping, readuce_lr],
    verbose = 1)